# **The ChipTracker Class**

This notebook provides information about the ```ChipTracker``` objects.

Run the following codeblocks to import the ```ChipTracker``` class into this notebook.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: d:\My Projects\Programming\Game Development\fivecarddraw


In [2]:
from fivecarddraw import ChipTracker

## **ChipTracker Functionality**

### **Initialisation**

The ```ChipTracker``` class is designed to run as a backend-subclass for ```Dealer``` objects. Upon initialisation:
* It creates attributes to track game parameters, such as the ante amount.
* It creates attributes to track the chipstacks and any contributions to the pot made by players.

More details can be found below.

### **Assigning chips to players**

The recommended way to assign chips to a player is using ```ChipTracker.AddChipsPlayer(name, amount)``` which correctly updates ```ChipTracker.players```.

In [3]:
player = "Brad"

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
print(f"Player Chipstacks: {tracker.ChipStacks()}")

Player Chipstacks: {'Brad': 500}


```ChipTracker.WithdrawChipsPlayer(name)``` can be used to delete all information about a player.

In [4]:
tracker.WithdrawChipsPlayer(player)
print(f"Player Chipstacks: {tracker.ChipStacks()}")

Player Chipstacks: {}


### **Contributing chips to the pot**

The recommended way to contribute chips to a player is using ```ChipTracker.BetChipsPlayer(name, amount)``` which correctly updates ```ChipTracker.players``` to track both chipstacks and contributions.

In [7]:
player = "Brad"

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
print(f"Player Chips before bet: {tracker.players}")
tracker.BetChipsPlayer(player, 100)
print(f"Player Chips after bet: {tracker.players}")

Player Chips before bet: {'Brad': {'stack': 500, 'contribution': 0}}
Player Chips after bet: {'Brad': {'stack': 400, 'contribution': 100}}


To ensure a player has bet a legal amount, the ```ChipTracker.ApproveBet(name, amount)``` can be used.

In [8]:
player = "Brad"
bet = 700

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
print(f"Player Chips before bet: {tracker.players}")
print(f"Bet to approve: {bet}")
print(f"Approval: {tracker.ApproveBet(player, 700)}")

Player Chips before bet: {'Brad': {'stack': 500, 'contribution': 0}}
Bet to approve: 700
Approval: False


### **Rewarding chips from a pot**

```ChipTracker.RewardChipsPlayer(name, amount)``` can be used to add an amount of chips to a player. `

**Warning:** ```ChipTracker.RewardChipsPlayer(name, amount)``` does not modify player contribution values, so it's recommended to clear contributions using ```ChipTracker.ClearPot()``` once all players have received their rewards.

In [12]:
player = "Brad"

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
tracker.AddChipsPlayer("Phil", 500)
tracker.BetChipsPlayer(player, 100)
tracker.BetChipsPlayer("Phil", 100)
print(f"Player Chips before rewarding: {tracker.players}")
tracker.RewardChipsPlayer(player, 200)
tracker.ClearPot()
print(f"Player Chips after rewarding, after pot is cleared: {tracker.players}")

Player Chips before rewarding: {'Brad': {'stack': 400, 'contribution': 100}, 'Phil': {'stack': 400, 'contribution': 100}}
Player Chips after rewarding, after pot is cleared: {'Brad': {'stack': 600, 'contribution': 0}, 'Phil': {'stack': 400, 'contribution': 0}}


In order to calculate rewards for players, ```ChipTracker.CalculateRewards(player_info)``` can be used which returns a ```dict``` of ```name : reward```  pairs. Rewards are calculated by inspecting player contributions, action statuses and hand ranks, taking into account split and side pots too. For more information, see the [dealer.ipynb](dealer.ipynb) notebook which outlines this procedure while describing the ```Dealer.Payout()``` method.

### **Classifying transactions**

```ChipTracker``` objects have the following methods that help classify transactions:
* ```ChipTracker.FeeStatus(name, amount)```
* ```ChipTracker.BetStatus(name, amount)```

#### **Paying Fees**

The ```ChipTracker.FeeStatus(name, amount)``` method checks a fee against a players chipstack, and returns a dictionary of indicators that inform an action to take.
* ```has_enough``` indicates wether the player can pay the fee.
* ```has_allin``` indicates wether the player needs to go allin to pay the fee.
* ```has_passed``` indicates wether there is a fee or not.


In [13]:
player = "Brad"
fee = 5

tracker = ChipTracker()
tracker.AddChipsPlayer(player, 500)
tracker.FeeStatus(player, fee)

{'has_enough': True, 'has_allin': False, 'has_passed': False}

#### **Making Actions**